In [16]:
import pandas as pd
import requests
import json


In [17]:
# Set the base URL
base_url = "https://data.cityofnewyork.us/resource/qgea-i56i.json?"
# Set the query URL
query_url = base_url + "$limit=50000"
response = requests.get(query_url).json()
# Create a DataFrame
original_df = pd.DataFrame(response)
# Display original df
original_df

,cmplnt_num,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_dt,cmplnt_to_tm,addr_pct_cd,rpt_dt,ky_cd,ofns_desc,pd_cd,...,station_name,vic_age_group,vic_race,vic_sex,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,transit_district
0,279760384,2023-12-30T00:00:00.000,16:24:00,2023-12-30T00:00:00.000,16:30:00,50,2023-12-31T00:00:00.000,361,OFF. AGNST PUB ORD SENSBLTY &,639,...,(null),25-44,BLACK,F,11272,48,5,40,33,NaN
1,279750942,2023-12-29T00:00:00.000,19:00:00,2023-12-30T00:00:00.000,10:00:00,109,2023-12-31T00:00:00.000,109,GRAND LARCENY,439,...,(null),65+,ASIAN / PACIFIC ISLANDER,M,14190,22,3,3,67,NaN
2,279752718,2023-12-31T00:00:00.000,00:01:00,2023-12-31T00:00:00.000,02:00:00,70,2023-12-31T00:00:00.000,121,CRIMINAL MISCHIEF & RELATED OF,267,...,(null),25-44,WHITE,M,17217,60,2,15,43,NaN
3,279760473,2023-12-31T00:00:00.000,17:50:00,2023-12-31T00:00:00.000,17:59:00,113,2023-12-31T00:00:00.000,250,CANNABIS RELATED OFFENSES,579,...,(null),UNKNOWN,UNKNOWN,E,24669,41,3,6,71,NaN
4,279749499,2023-12-31T00:00:00.000,09:25:00,2023-12-31T00:00:00.000,09:30:00,109,2023-12-31T00:00:00.000,348,VEHICLE AND TRAFFIC LAWS,916,...,(null),25-44,ASIAN / PACIFIC ISLANDER,F,14192,22,3,20,67,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,278208943,2023-11-27T00:00:00.000,16:20:00,2023-11-27T00:00:00.000,16:23:00,78,2023-11-28T00:00:00.000,341,PETIT LARCENY,333,...,(null),UNKNOWN,UNKNOWN,D,17619,14,2,27,50,NaN
49996,278214049,2023-11-28T00:00:00.000,19:02:00,2023-11-28T00:00:00.000,19:26:00,108,2023-11-28T00:00:00.000,578,HARRASSMENT 2,637,...,(null),25-44,BLACK,M,16858,53,3,33,66,NaN
49997,278164135,2023-11-27T00:00:00.000,18:00:00,2023-11-28T00:00:00.000,07:45:00,49,2023-11-28T00:00:00.000,109,GRAND LARCENY,457,...,(null),25-44,BLACK HISPANIC,M,11271,59,5,12,32,NaN
49998,278214045,2023-11-25T00:00:00.000,00:09:00,2023-11-25T00:00:00.000,00:21:00,104,2023-11-28T00:00:00.000,578,HARRASSMENT 2,637,...,(null),45-64,WHITE HISPANIC,M,15310,54,3,34,62,NaN
